# Problem Set 7

Ariel Boyarsky

aboyarsky@uchciago.edu

In [256]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold

1. Classifier “horse” race (10 points). For this problem, you will use the 397
observations from the Auto.csv dataset.1 This dataset includes 397 observations
on miles per gallon (mpg), number of cylinders (cylinders), engine displacement
(displacement), horsepower (horsepower), vehicle weight (weight), acceleration
(acceleration), vehicle year (year), vehicle origin (origin), and
vehicle name (name). We will study the factors that make miles per gallon
high or low. Create a binary variable mpg high that equals 1 if mpg high≥
median(mpg high) and equals either 0 if mpg high< median(mpg high).

In [257]:
dta = pd.read_csv("Auto.csv", na_values='?')
dta = dta.dropna()

In [258]:
def mpg_level(mpg):
    med = np.median(dta.mpg)
    if mpg >= med:
        return 1
    else:
        return 0
    
dta['mpg_high'] = dta.apply(lambda row: mpg_level(row['mpg']), axis =1)

a. Use sklearn.linear model.LogisticRegression to fit a logistic model
of mpg high on features number of cylinders (cyl), engine displacement
(dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehicle
year (yr), vehicle origin (orgn). Make sure to include a constant term.
Fit the model using k-fold cross validation with k = 4 folds.

Report the MSE of the model as the average MSE across the k = 4 test
sets, and report the error rates for each category of mpg high as the average
error rate for that category across the k = 4 test sets.


In [259]:
dta.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0


In [260]:
X = dta[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = dta['mpg_high'].values

In [261]:
k=4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
y_test_vec = np.array([])
y_pred_vec = np.array([])
e_0 = np.zeros(k)
e_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_test_vec = np.append(y_test_vec, y_test)
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    y_pred_vec = np.append(y_pred_vec, y_pred)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    err0 = ((y_pred==0)*(np.abs(y_test-y_pred))).sum() / (y_pred==0).sum()
    e_0[k_ind] = (err0)
    err1 = ((y_pred==1)*(np.abs(y_test-y_pred))).sum() / (y_pred==1).sum()
    e_1[k_ind] = (err1)
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('Error rate (low, 0): ', err0)
    print('Error rate (high, 1): ', err1)
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print(' avg low error rate: ', e_0.mean(), '\n AVG high error rate: ', e_1.mean(), '\n Average MSE: ', MSE_vec_kf.mean())

k index= 0
MSE for test set 0  is 0.0918367346939
Error rate (low, 0):  0.0576923076923
Error rate (high, 1):  0.130434782609

              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

k index= 1
MSE for test set 1  is 0.102040816327
Error rate (low, 0):  0.122448979592
Error rate (high, 1):  0.0816326530612

              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

k index= 2
MSE for test set 2  is 0.132653061224
Error rate (low, 0):  0.152173913043
Error rate (high, 1):  0.115384615385

              precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0

b. Use sklearn.ensemble.RandomForestClassifier to fit a random forest
model of mpg high on max features=2 out of the seven possible features
used in part (a). Set n estimators=20, set bootstrap=True, set
oob score=True, and set random state=25. Report the MSE of the random
forest model as the MSE from the .oob prediction object, and report
the error rates for each category of mpg high from the .oob prediction
object.


In [262]:
forest = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
forest.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=True, random_state=25,
            verbose=0, warm_start=False)

In [263]:
forest.score(X, y)
MSE_df = pd.DataFrame({'ypred': forest.oob_decision_function_.T[1], 'y': y})
MSE_df['ypred'] = MSE_df['ypred'].apply(lambda x: 1 if int(x>=0.5) else 0)
MSE = mean_squared_error(MSE_df['y'], MSE_df['ypred'])
MSE_0 = mean_squared_error(MSE_df['y'].where(MSE_df.ypred<0.5).dropna(), MSE_df['ypred'].where(MSE_df.ypred<0.5).dropna())
MSE_1 = mean_squared_error(MSE_df['y'].where(MSE_df.ypred>0.5).dropna(), MSE_df['ypred'].where(MSE_df.ypred>0.5).dropna())
print('MSE=', MSE)
print('MSE for low category (0) = ', MSE_0)
print('MSE for low category (0) = ', MSE_1)

MSE= 0.0714285714286
MSE for low category (0) =  0.0578947368421
MSE for low category (0) =  0.0841584158416


c.Use sklearn.svm.SVC to fit a support vector machines model of mpg high
with a Gaussian radial basis function kernel kernel=’rbf’ on the seven
features used in part (a). Set the penalty parameter to C=1 and set
gamma=0.2. Fit the model using k-fold cross validation with k = 4 folds
exactly as in part (a).


Report the MSE of the model as the average MSE across the k = 4 test
sets, and report the error rates for each category of mpg high as the average
error rate for that category across the k = 4 test sets

In [264]:
k=4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
y_test_vec = np.array([])
y_pred_vec = np.array([])
e_0 = np.zeros(k)
e_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_test_vec = np.append(y_test_vec, y_test)
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    y_pred_vec = np.append(y_pred_vec, y_pred)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    err0 = ((y_pred==0)*(np.abs(y_test-y_pred))).sum() / (y_pred==0).sum()
    e_0[k_ind] = (err0)
    err1 = ((y_pred==1)*(np.abs(y_test-y_pred))).sum() / (y_pred==1).sum()
    e_1[k_ind] = (err1)
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('Error rate (low, 0): ', err0)
    print('Error rate (high, 1): ', err1)
    print('\n',classification_report(y_test, y_pred, digits=3))
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print(' avg low error rate: ', e_0[~np.isnan(e_0)].mean(), '\n AVG high error rate: ', e_1[~np.isnan(e_1)].mean(), '\n Average MSE: ', MSE_vec_kf.mean())

k index= 0
MSE for test set 0  is 0.540816326531
Error rate (low, 0):  0.0
Error rate (high, 1):  0.552083333333

              precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

k index= 1
MSE for test set 1  is 0.520408163265
Error rate (low, 0):  0.520408163265
Error rate (high, 1):  nan

              precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

k index= 2
MSE for test set 2  is 0.520408163265
Error rate (low, 0):  0.53125
Error rate (high, 1):  0.0

              precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

k index= 3
MSE fo

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in longlong_scalars
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


d. Which of the above three models do you think is the best predictor of
mpg high? Why?

Based on the above results it is clear that the the *Random Forest is the best model*. This is because the error rates for this model are the lowest. The MSE is also lowest on this model.The SVM is the least accurate of the three and the logistic regression model is the next best after the random forest. Notice that MSE on the SVM is $\approx0.507$ while on the Logit model it is $\approx0.107$ and on the random forest it is $\approx0.071$. This trend also continues in error rates in which  the random forest has by farthe lowest high and low mpg error rates.